In [ ]:
pip install pycryptodome

In [64]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes

---

Digtal Signatures and verification

In [65]:
def sign_message(message, private_sig_key):
    h = SHA256.new(message)
    return pkcs1_15.new(private_sig_key).sign(h)

In [66]:
def verify_signature(message, signature, public_sig_key):
    key = RSA.import_key(public_sig_key)
    h = SHA256.new(message)
    try:
        pkcs1_15.new(key).verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

Asymmetric Encryption using RSA key pairs

In [67]:
def aenc(message, public_key):
    key = RSA.import_key(public_key)
    cipher = PKCS1_OAEP.new(key)
    return cipher.encrypt(message)

In [68]:
def adec(ciphertext, private_key):
    decipher = PKCS1_OAEP.new(private_key)
    plaintext = decipher.decrypt(ciphertext)
    return plaintext

Symmetric Encryption using AES

In [69]:
def senc(message, session_key):
    cipher_CTR = AES.new(session_key, AES.MODE_CTR)
    ciphertext = cipher_CTR.encrypt(message)
    return ciphertext, cipher_CTR.nonce

In [70]:
def sdec(ciphertext, session_key, nonce):
    decipher_CTR = AES.new(session_key, AES.MODE_CTR, nonce=nonce)
    plaintext = decipher_CTR.decrypt(ciphertext)
    return plaintext

Hashes for data integrity

In [71]:
def calculate_hash(message, key):
    h = HMAC.new(key, digestmod=SHA256)
    h.update(message)
    return h.hexdigest()

---

In [72]:
class Phone:
    private_sig_key = RSA.generate(1024)
    public_sig_key = private_sig_key.public_key().export_key()
    private_key = RSA.generate(1024)
    public_key = private_key.public_key().export_key()

In [73]:
class Server:
    private_sig_key = RSA.generate(1024)
    public_sig_key = private_sig_key.public_key().export_key()
    private_key = RSA.generate(1024)
    public_key = private_key.public_key().export_key()

    @classmethod
    def set_secret_key_IoT(cls, value):
        cls.secret_key_IoT = value

In [74]:
class IoT_Device:
    private_sig_key = RSA.generate(1024)
    public_sig_key = private_sig_key.public_key().export_key()
    private_key = RSA.generate(1024)
    public_key = private_key.public_key().export_key()

    @classmethod
    def set_secret_key_IoT(cls, value):
        cls.secret_key_IoT = value

In [75]:
preshared_key = get_random_bytes(16) # preshared key
IoT_Device.set_secret_key_IoT(preshared_key)
Server.set_secret_key_IoT(preshared_key)

---

In [76]:
server_accepts_phone = False
server_terminates_phone = False
server_accepts_iot_device = False
server_terminates_iot_device = False
phone_accepts = False
phone_terminates = False
iot_device_accepts = False
iot_device_terminates = False

All public keys are made accessible to everyone 

In [77]:
pk_Phone = Phone.public_key
spk_Phone = Phone.public_sig_key
pk_Server = Server.public_key
spk_Server = Server.public_sig_key

Phone initiates needham protocol with the server

In [78]:
def Phone_1():
    nonce_21 = get_random_bytes(16)
    m21 = aenc(nonce_21, pk_Server)
    m21_sig = sign_message(m21, Phone.private_sig_key)
    return m21, m21_sig, nonce_21

The server receives the nonce, generates a second, and sends it back to the Phone.

In [80]:
def Server_1(m21, m21_sig):
    if(verify_signature(m21, m21_sig, spk_Phone)):
        nonce_21 = adec(m21, Server.private_key)
        nonce_22 = get_random_bytes(16)
        global server_accepts_phone
        server_accepts_phone = True
        m22 = aenc(nonce_21 + nonce_22, pk_Phone)
        m22_sig = sign_message(m22, Server.private_sig_key)
        return m22, m22_sig, nonce_22

The phone tests the sent nonces. If the original nonce matches, then the needham protocol succeeds.

In [79]:
def Phone_2(m22, m22_sig, NONCE_21):
    if(verify_signature(m22, m22_sig, spk_Server)):
        msg = adec(m22, Phone.private_key)
        nonce_21 = msg[:len(msg) // 2]
        nonce_22 = msg[len(msg) // 2:]
        if(NONCE_21 == nonce_21):
            global phone_accepts
            phone_accepts = True
            m23 = aenc(nonce_22, pk_Server)
            m23_sig = sign_message(m23, Phone.private_sig_key)
            global phone_terminates
            phone_terminates = True
            return m23, m23_sig
        else: 
            print("Phone does not accept")

The server then completes it's half of the needham protocol and establishes a connection with the IoT device.

In [81]:
def Server_2(m23, m23_sig, NONCE_22):
    if(verify_signature(m23, m23_sig, spk_Phone)):
        nonce_22 = adec(m23, Server.private_key)
        if(NONCE_22 == nonce_22):
            global server_terminates_phone
            server_terminates_phone = True
            nonce_11 = get_random_bytes(16)
            control_data = get_random_bytes(16)
            m11_1, nonce_aes_1 = senc(nonce_11, Server.secret_key_IoT)
            m11_2, nonce_aes_2 = senc(control_data, Server.secret_key_IoT)
            h11_1 = calculate_hash(m11_1, Server.secret_key_IoT)
            h11_2 = calculate_hash(m11_2, Server.secret_key_IoT)
            return m11_1, m11_2, h11_1, h11_2, nonce_11, nonce_aes_1, nonce_aes_2
        else:
            print("Server does not accept")

Using Hashes to complete the needham protocol, the server and the IoT device send each other control and output data.

In [83]:
def IoT_Device_1(m11_1, m11_2, h11_1, h11_2, nonce_aes_1, nonce_aes_2):
    if(calculate_hash(m11_1, IoT_Device.secret_key_IoT) == h11_1 and calculate_hash(m11_2, IoT_Device.secret_key_IoT) == h11_2):
        nonce_11 = sdec(m11_1, IoT_Device.secret_key_IoT, nonce_aes_1)
        control_data = sdec(m11_2, IoT_Device.secret_key_IoT, nonce_aes_2)
        global iot_device_accepts
        iot_device_accepts = True
        nonce_12 = get_random_bytes(16)
        output_data = get_random_bytes(16)
        m12_1, nonce_aes_3 = senc(nonce_11 + nonce_12, IoT_Device.secret_key_IoT)
        m12_2, nonce_aes_4 = senc(output_data, IoT_Device.secret_key_IoT)
        h12_1 = calculate_hash(m12_1, IoT_Device.secret_key_IoT)
        h12_2 = calculate_hash(m12_2, IoT_Device.secret_key_IoT)
        global server_terminates_iot_device
        server_terminates_iot_device = True
        return m12_1, m12_2, h12_1, h12_2, nonce_12, nonce_aes_3, nonce_aes_4
    else:
        print("IoT does not accept")

In [82]:
def Server_3(m12_1, m12_2, h12_1, h12_2, NONCE_11, nonce_aes_3, nonce_aes_4):
    if(calculate_hash(m12_1, Server.secret_key_IoT) == h12_1 and calculate_hash(m12_2, Server.secret_key_IoT) == h12_2):
        msg = sdec(m12_1, Server.secret_key_IoT, nonce_aes_3)
        nonce_11 = msg[:len(msg) // 2]
        nonce_12 = msg[len(msg) // 2:]
        if(NONCE_11 == nonce_11):
            global server_accepts_iot_device
            server_accepts_iot_device = True
            m13 , nonce_aes_5= senc(nonce_12, Server.secret_key_IoT)
            h13 = calculate_hash(m13, Server.secret_key_IoT)
            global server_terminates_iot_device
            server_terminates_iot_device = True
            return m13, h13, nonce_aes_5
    else:
        print("Server does not accept")

In [84]:
def IoT_Device_2(m13, h13, nonce_aes_5, NONCE_12):
    if(calculate_hash(m13, IoT_Device.secret_key_IoT) == h13):
        nonce_12 = sdec(m13, IoT_Device.secret_key_IoT, nonce_aes_5)
        if(NONCE_12 == nonce_12):
            global iot_device_terminates
            iot_device_terminates = True
    else:
        print("IoT does not accept")

In [85]:
m21, m21_sig, NONCE_21 = Phone_1()

m22, m22_sig, NONCE_22 = Server_1(m21, m21_sig)
print("server_accepts = ", server_accepts_phone)

m23, m23_sig = Phone_2(m22, m22_sig, NONCE_21)
print("phone_accepts = ", phone_accepts)
print("phone_terminates = ", phone_terminates)

m11_1, m11_2, h11_1, h11_2, NONCE_11, nonce_aes_1, nonce_aes_2 = Server_2(m23, m23_sig, NONCE_22)

m12_1, m12_2, h12_1, h12_2, NONCE_12, nonce_aes_3, nonce_aes_4 = IoT_Device_1(m11_1, m11_2, h11_1, h11_2, nonce_aes_1, nonce_aes_2)
print("iot_device_accepts = ", iot_device_accepts)

m13, h13, nonce_aes_5 = Server_3(m12_1, m12_2, h12_1, h12_2, NONCE_11, nonce_aes_3, nonce_aes_4)
print("server_accepts_iot_device = ", server_accepts_iot_device)
print("server_terminates_iot_device = ", server_terminates_iot_device)

IoT_Device_2(m13, h13, nonce_aes_5, NONCE_12)
print("iot_device_terminates = ", iot_device_terminates)

server_accepts =  True
phone_accepts =  True
phone_terminates =  True
iot_device_accepts =  True
server_accepts_iot_device =  True
server_terminates_iot_device =  True
iot_device_terminates =  True
